<a href="https://colab.research.google.com/github/amitadhainje/LearningBERT/blob/master/Exploring_TF_Model_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install -q tensorflow-model-analysis==0.22.0

import tensorflow as tf
import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.__version__))

TFMA version: 0.22.0


In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
!mv /content/'IMDB Dataset.csv' /content/drive/'My Drive'/'Colab Notebooks'/NLP/
!ls -ltr /content/drive/'My Drive'/'Colab Notebooks'/NLP


mv: cannot stat '/content/IMDB Dataset.csv': No such file or directory
total 931342
-rw-------  1 root root 346919801 Nov 21  2011  europarl-v7.fr-en.fr
-rw-------  1 root root 301523301 Nov 21  2011  europarl-v7.fr-en.en
-rw-------  1 root root      5010 Mar 16 06:59  BERT_With_TF2.ipynb
-rw-------  1 root root     74326 Mar 19 05:17  test.csv
-rw-------  1 root root 238803811 Mar 19 06:23  train.csv
-rw-------  1 root root     29990 Mar 19 12:16  BERT_SentimentAnalysis.ipynb
-rw-------  1 root root     34459 Mar 20 11:56  Bert_SentimentAnalysis_Part_2.ipynb
-rw-------  1 root root     66386 Apr  6 10:05  Coursera-Natural-Language-Processing-WithTF-Week-1.ipynb
-rw-------  1 root root     10120 Apr  7 07:57  Word2Vec-Exploration.ipynb
-rw-------  1 root root      6970 Apr 21 10:37  Exploring-BERT-VocbFile.ipynb
-rw-------  1 root root       110 May  1 09:04  nonbreaking_prefix.en
-rw-------  1 root root       121 May  1 09:05  nonbreaking_prefix.fr
drwx------  2 root root      4096 Ma

In [0]:
from __future__ import print_function

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, LSTM
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/IMDB Dataset.csv', nrows=10000)
df.sentiment = df.sentiment.map({ 'positive' : 1, 'negative' : 0})
#df.loc[df['sentiment'] == 'positive', sentiment] = 1
from sklearn.model_selection import train_test_split

texts_train, texts_test, y_train, y_test = train_test_split(df["review"].values, df['sentiment'].values, random_state=42)

In [0]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

tokenizer = Tokenizer(num_words=max_features)
trainingData = tokenizer.fit_on_texts(texts_train)

train_data = tokenizer.texts_to_sequences(texts_train)
test_data = tokenizer.texts_to_sequences(texts_test)

padded_train_data = pad_sequences(train_data, maxlen=80, padding="post")
padded_test_data = pad_sequences(test_data, maxlen=80, padding="post")

In [35]:
batch_size = 64
max_features = len(tokenizer.word_index) + 1

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features, 128))
model.add(tf.keras.layers.LSTM(128, dropout=0.5, recurrent_dropout=0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
print (model.summary())
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

model.fit(padded_train_data, y_train, epochs=2, batch_size=batch_size)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         6727552   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 6,859,265
Trainable params: 6,859,265
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
118/118 [==============================] - 43s 362ms/step - loss: 0.5695 - accuracy: 0.6909
Epoch 2/2
118/118 [==============================] - 42s 357ms/step - loss: 0.3002 - accuracy: 0.8824


In [36]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(padded_test_data, y_test, batch_size=64)
print('test loss, test acc:', results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(padded_test_data[:3])
print('predictions shape:', predictions.shape)


# Evaluate on test data
40/40 [==============================] - 2s 41ms/step - loss: 0.3859 - accuracy: 0.8324
test loss, test acc: [0.38586729764938354, 0.8324000239372253]

# Generate predictions for 3 samples
predictions shape: (3, 1)


In [37]:
!mkdir /content/saved_model
model.save("/content/saved_model/")

mkdir: cannot create directory ‘/content/saved_model’: File exists
INFO:tensorflow:Assets written to: /content/saved_model/assets


INFO:tensorflow:Assets written to: /content/saved_model/assets


In [0]:
#review_list = tf.train.BytesList(value=texts_train)
review_list = []
sentiment_list = []

for x in range(0, len(padded_train_data)):
    review_list.append(tf.train.Feature(int64_list=tf.train.Int64List(value=padded_train_data[x])))
    sentiment_list.append(tf.train.Feature(int64_list=tf.train.Int64List(value=[y_train[x]])))

In [0]:
reviews = tf.train.FeatureList(feature=review_list)
sentiments = tf.train.FeatureList(feature=sentiment_list)

In [0]:
movie_reviews = tf.train.FeatureLists(feature_list={
    'Reviews': reviews,
    'Sentiments': sentiments
})

In [0]:
# Create the SequenceExample
example = tf.train.SequenceExample(feature_lists=movie_reviews)
with tf.io.TFRecordWriter('reviews.tfrecord') as writer:
    writer.write(example.SerializeToString())

In [42]:
!mkdir /content/output

mkdir: cannot create directory ‘/content/output’: File exists


In [28]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [48]:
from google.protobuf import text_format

eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # This assumes a serving model with a "serving_default" signature.
    label_key: "dense_2"
    example_weight_key: "weight"
  }
  ## Post export metric information
  metrics_specs {
    # This adds AUC and as a post training metric. If the model has built in
    # training metrics which also contains AUC, this metric will replace it.
    metrics { class_name: "AUC" }
    # ... other post training metrics ...

    # Plots are also configured here...
    metrics { class_name: "ConfusionMatrixPlot" }
  }

""", tfma.EvalConfig())

eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path='/content/saved_model', eval_config=eval_config)

eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model,
    eval_config=eval_config,
    # This assumes your data is a TFRecords file containing records in the
    # tf.train.Example format.
    data_location='/content/reviews.tfrecord',
    output_path='/content/output')

tfma.view.render_slicing_metrics(eval_result)

AttributeError: ignored

In [47]:
!saved_model_cli show --dir "/content/saved_model/" --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['embedding_2_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1)
      name: serving_default_embedding_2_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_2'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [44]:
import pyarrow as pa
print(pa.__version__)

0.15.1


In [0]:
!pip install pyarrow==0.15.1